# Import

In [1]:
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
import optuna
from joblib import Parallel, delayed
from utils.Loader import NEUDataset
from utils.Perspectiver import Perspectiver
from source.Prototype1 import Prototype1

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import multiprocessing
import torch
from mpl_toolkits.mplot3d import Axes3D
from skimage.restoration import denoise_wavelet
import random
import math
from PIL import Image
from collections import deque
import numpy as np
from scipy.ndimage import maximum_filter, minimum_filter, label, generate_binary_structure
from concurrent.futures import ProcessPoolExecutor, as_completed
from scipy.ndimage import label as ndi_label, binary_dilation

/home/lingfeng/Desktop/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Init Variables

In [2]:
model = Prototype1(num_attention_heads=16)
if torch.cuda.is_available():
    print("Using cuda cores")
    model.cuda()

dataset = NEUDataset(set="train", seed=555, scale=0.5)

Using cuda cores


# Hyper parameters

In [3]:
LEARNING_RATE = 0.0001
EPOCH = 10
BATCH_SIZE = 512
THREADS = 8

# Training process

In [4]:
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
dataloader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=THREADS)

In [5]:
for epoch in range(EPOCH):
    model.train()
    total_loss = 0
    num_batches = 0

    # Desempaquetar ignorando 'labels'
    for images, _, best_parameters in dataloader:
        images = images.to('cuda')
        best_parameters = best_parameters.to('cuda')

        # Forward: el modelo predice sp y sr
        pred_reg = model(images)  # salida de regresión

        # Calcular la pérdida de regresión
        loss = criterion(pred_reg, best_parameters)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        num_batches += 1

    avg_loss = total_loss / num_batches
    print(f"Epoch [{epoch + 1}/{EPOCH}], Loss: {avg_loss:.4f}")


Epoch [1/10], Loss: 1868.8723
Epoch [2/10], Loss: 1868.8340
Epoch [3/10], Loss: 1868.7948
Epoch [4/10], Loss: 1868.7566
Epoch [5/10], Loss: 1868.7166
Epoch [6/10], Loss: 1868.6747
Epoch [7/10], Loss: 1868.6309
Epoch [8/10], Loss: 1868.5831
Epoch [9/10], Loss: 1868.5320
Epoch [10/10], Loss: 1868.4827


In [6]:
# Guardar solo la red neuronal en un archivo .pth
torch.save(model.state_dict(), "h1.pth")
print("Modelo guardado exitosamente en h1.pth")

Modelo guardado exitosamente en h1.pth
